In [1]:
from pathlib import Path

In [30]:
path = Path('data/ffhq')   #/home/dylee/taming-transformers/data/ffhq

In [32]:
path_list=list(path.glob('**/*.png'))

In [33]:
file_name_list = [path.stem for path in path_list ]

In [35]:
file_name_list

1000

In [38]:
with open('ffhqtrain_sub.txt', 'w') as f:
    for file_name in file_name_list[:800]:
        f.write(file_name + '.png' + '\n')

In [39]:
with open('ffhqvalidation_sub.txt', 'w') as f:
    for file_name in file_name_list[800:]:
        f.write(file_name + '.png' + '\n')

# MimicCXR

In [1]:
from pathlib import Path
import pandas as pd
import csv
import os

In [2]:
root = Path('/home/edlab/dylee/physionet.org/files/mimic-cxr-jpg/2.0.0')

In [3]:
split_file_dir = root / 'mimic-cxr-2.0.0-split.csv'

In [4]:
str(split_file_dir)

'/home/edlab/dylee/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-split.csv'

In [35]:
# The dataset contains 377,110 JPG format images
f = open(str(split_file_dir), 'r')
rdr = csv.reader(f)

trainset = []
valset = []
testset = []
others = []
num = 0
for i, line in enumerate(rdr):
    if i == 0: continue
    subject_id = 'p'+line[2]
    study_id = 's'+line[1]
    filename = line[0]+'.jpg'
    split = line[3]

    if split == 'train':
        trainset.append(
            os.path.join( subject_id[:3], subject_id, study_id, filename )
        )
    elif split == 'validate':
        valset.append(
            os.path.join( subject_id[:3], subject_id, study_id, filename )
        )
    elif split == 'test':
        testset.append(
            os.path.join( subject_id[:3], subject_id, study_id, filename )
        )
    else: 
        others.append(
            line
        )
    num = num + 1
f.close()


In [36]:
len(trainset)

368960

In [37]:
len(valset)

2991

In [38]:
len(testset)

5159

In [39]:
len(trainset) + len(valset) + len(testset)

377110

In [40]:
num == (len(trainset) + len(valset) + len(testset))

True

In [41]:
num

377110

In [44]:
with open('mimiccxrtrain.txt', 'w')as f:
    for i in range(len(trainset)):
        line = trainset[i] + '\n'
        f.write(line)

In [45]:
devset = valset + testset

In [46]:
len(devset)

8150

In [47]:
with open('mimiccxrvalidation.txt', 'w')as f:
    for i in range(len(devset)):
        line = devset[i] + '\n'
        f.write(line)

# Resizing

In [1]:
from tqdm import tqdm
import numpy as np
import albumentations
from PIL import Image
import os
root = '/home/edlab/dylee/physionet.org/files/mimic-cxr-jpg/2.0.0/files'
newroot = '/home/edlab/dylee/physionet.org/files/mimic-cxr-jpg/2.0.0/resized'

In [2]:
f = open('data/mimiccxrtrain.txt', 'r')
trainfiles = f.read().splitlines()    # f.readlines()하면 맨 뒤 \n도 붙어 있는데 이건 없네
f.close()

f = open('data/mimiccxrvalidation.txt', 'r')
valfiles = f.read().splitlines()    # f.readlines()하면 맨 뒤 \n도 붙어 있는데 이건 없네
f.close()

In [3]:
rescaler = albumentations.SmallestMaxSize(max_size = 256)    # Rescale an image so that minimum side is equal to max_size, keeping the aspect ratio of the initial image.
cropper = albumentations.CenterCrop(height=256, width=256)
preprocessor = albumentations.Compose([rescaler, cropper])   # input으로 numpy array만 받는다

In [4]:
def preprocess_image(image_path):
    image = Image.open(image_path)   # PIL format
    if not image.mode == "RGB":
        image = image.convert("RGB")
    image = np.array(image).astype(np.uint8)         # value 0 ~ 255  [H, W, C]
    image = preprocessor(image=image)["image"]  # albumentations의 output: {'image': numpy.ndarray(256,256,3)}  uint8
    return image   # ndarray (256, 256, 3)  uint8

In [ ]:
for filepath in tqdm(trainfiles):
    image = preprocess_image( os.path.join(root, filepath) )  # ndarray (256, 256, 3)  uint8
    newpath = os.path.join(newroot, filepath)
    os.makedirs( os.path.split(newpath)[0], exist_ok=True )   # os.path.split(path): 폴더부분과 파일부분을 서로 잘라준다. os.path.join의 반대
    Image.fromarray(image).save(newpath)

# Checking

In [4]:
from pathlib import Path

In [5]:
path = Path('/home/edlab/dylee/physionet.org/files/mimic-cxr-jpg/2.0.0/files')

In [6]:
path_list=list(path.glob('**/*.jpg'))

In [7]:
len(path_list)

377110

In [8]:
path = Path('/home/edlab/dylee/physionet.org/files/mimic-cxr-jpg/2.0.0/resized')

In [9]:
path_list=list(path.glob('**/*.jpg'))

In [10]:
len(path_list)

377110